In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras import regularizers

2024-02-09 23:49:05.311265: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-09 23:49:05.336593: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 23:49:05.336623: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 23:49:05.337451: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-09 23:49:05.342129: I tensorflow/core/platform/cpu_feature_guar

In [2]:
df1 = pd.read_csv("book1.csv")
df2 = pd.read_csv("book.csv")

df = pd.concat([df1, df2])



,label,text
0,medical,Medical History Form\n\nPatient Information:\n...
1,medical,Physical Examination Report\n\nPatient Informa...
2,medical,Laboratory Test Results (Blood Test)\n\nPatien...
3,medical,Radiology Report (X-ray)\n\nPatient Informatio...
4,medical,Prescription Record\n\nPatient Information:\n\...
...,...,...
95,government,Public Policy Report\n\nTitle: National Climat...
96,government,Environmental Impact Statement (EIS)\n\nTitle:...
97,government,Public Health Advisory\n\nTitle: Centers for D...
98,government,Government Audit Report\n\nTitle: Internal Re...


In [14]:
df['label'].value_counts()

label
medical        21
financial      21
educational    21
business       21
news           21
technical      21
creative       21
scientific     21
government     21
legal          20
legal           1
Name: count, dtype: int64

In [4]:
def preprocess(text):
    text = re.sub("[_\n]", " ", text)
    return re.sub(r" +", "", text)


df["text"] = df["text"].apply(preprocess)

In [5]:
bert_preprocess = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
)
bert_encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"
)

2024-02-09 22:54:20.805754: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-09 22:54:20.843255: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-09 22:54:20.843433: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

In [7]:
df

,label,text
0,6,MedicalHistoryFormPatientInformation:Name:Edwa...
1,6,PhysicalExaminationReportPatientInformation:Na...
2,6,LaboratoryTestResults(BloodTest)PatientInforma...
3,6,RadiologyReport(X-ray)PatientInformation:Name:...
4,6,PrescriptionRecordPatientInformation:Name:Rohi...
...,...,...
95,4,PublicPolicyReportTitle:NationalClimateChangeP...
96,4,EnvironmentalImpactStatement(EIS)Title:Propose...
97,4,PublicHealthAdvisoryTitle:CentersforDiseaseCon...
98,4,GovernmentAuditReportTitle:InternalRevenueServ...


In [8]:
df = df.sample(frac=1, random_state=30)
train_data, temp_data = train_test_split(
    df, test_size=0.2, stratify=df["label"], random_state=30
)
dev_data, test_data = train_test_split(
    temp_data, test_size=0.5, stratify=temp_data["label"], random_state=30
)

In [9]:
import numpy as np

X_train = np.array((train_data["text"].tolist()))
y_train = np.array(to_categorical(train_data["label"].tolist(), num_classes=10))
X_dev = np.array((dev_data["text"].tolist()))
y_dev = np.array(to_categorical(dev_data["label"].tolist(), num_classes=10))
X_test = np.array((test_data["text"].tolist()))
y_test = np.array(to_categorical(test_data["label"].tolist(), num_classes=10))

In [10]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

In [11]:
lambda_reg = 0.01

In [12]:
dropout_layer1 = tf.keras.layers.Dropout(0.1)(outputs["pooled_output"])
dense_layer1 = tf.keras.layers.Dense(
    512, activation="relu", kernel_regularizer=regularizers.l2(lambda_reg)
)(dropout_layer1)
dropout_layer2 = tf.keras.layers.Dropout(0.1)(dense_layer1)
dense_layer2 = tf.keras.layers.Dense(
    512, activation="relu", kernel_regularizer=regularizers.l2(lambda_reg)
)(dropout_layer2)
output = tf.keras.layers.Dense(10, activation="softmax", name="output")(dense_layer2)
model = tf.keras.Model(inputs=[text_input], outputs=output)

In [13]:
batch_size = 32

In [14]:
STEPS_PER_EPOCH = len(X_train) // batch_size
#
custom_lr = PolynomialDecay(
    initial_learning_rate=0.01, decay_steps=STEPS_PER_EPOCH, end_learning_rate=0.0001
)
# # Now you can use STEPS_PER_EPOCH in your learning rate schedule
# custom_lr = tf.keras.optimizers.schedules.InverseTimeDecay(
#     0.001, decay_steps=STEPS_PER_EPOCH * 1000, decay_rate=1, staircase=False
# )

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=custom_lr),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

2024-02-09 22:54:25.830559: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [16]:
early_stopping = EarlyStopping(monitor="val_loss", patience=3)
checkpoint = ModelCheckpoint(
    "best_model.h5", monitor="val_accuracy", verbose=1, save_best_only=True, mode="max"
)
tensorboard = TensorBoard(log_dir="./logs")

In [17]:
model.evaluate(X_train, y_train)

5/5 [==============================] - 2s 264ms/step - loss: 13.6331 - accuracy: 0.1000


[13.633079528808594, 0.10000000149011612]

In [18]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 1s 615ms/step - loss: 13.6398 - accuracy: 0.1000


[13.63978099822998, 0.10000000149011612]

In [19]:
model.evaluate(X_dev, y_dev)

1/1 [==============================] - 0s 181ms/step - loss: 13.6282 - accuracy: 0.1000


[13.62816333770752, 0.10000000149011612]

In [20]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_dev, y_dev),
    epochs=300,
    batch_size=batch_size,
    callbacks=[early_stopping],
)

Epoch 1/300


2024-02-09 22:54:33.393100: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f481c13de10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-09 22:54:33.393124: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-02-09 22:54:33.396316: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-09 22:54:33.414401: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1707499473.475863   34453 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/5 [==============================] - 5s 325ms/step - loss: 18.9395 - accuracy: 0.1187 - val_loss: 12.5155 - val_accuracy: 0.1000
Epoch 2/300
5/5 [==============================] - 2s 318ms/step - loss: 12.3803 - accuracy: 0.1063 - val_loss: 12.0105 - val_accuracy: 0.1500
Epoch 3/300
5/5 [==============================] - 2s 316ms/step - loss: 11.8563 - accuracy: 0.1312 - val_loss: 11.4402 - val_accuracy: 0.2000
Epoch 4/300
5/5 [==============================] - 2s 313ms/step - loss: 11.3038 - accuracy: 0.1312 - val_loss: 10.9038 - val_accuracy: 0.1000
Epoch 5/300
5/5 [==============================] - 2s 318ms/step - loss: 10.7249 - accuracy: 0.1500 - val_loss: 10.4294 - val_accuracy: 0.2500
Epoch 6/300
5/5 [==============================] - 2s 314ms/step - loss: 10.3162 - accuracy: 0.1813 - val_loss: 9.9992 - val_accuracy: 0.2500
Epoch 7/300
5/5 [==============================] - 2s 314ms/step - loss: 9.9197 - accuracy: 0.1937 - val_loss: 9.6192 - val_accuracy: 0.2000
Epoch 8/300
5/

In [21]:
model.save("pdf_classifier")

INFO:tensorflow:Assets written to: pdf_classifier/assets


INFO:tensorflow:Assets written to: pdf_classifier/assets


In [22]:
model.evaluate(X_test, y_test)
y = model.predict(X_test)
predicted_indices = np.argmax(y, axis=-1)
original_labels = label_encoder.inverse_transform(predicted_indices)

1/1 [==============================] - 0s 449ms/step


In [23]:
print(original_labels)

['scientific' 'medical' 'medical' 'government' 'scientific' 'creative'
 'scientific' 'business' 'business' 'creative' 'business' 'educational'
 'technical' 'business' 'financial' 'government' 'government' 'financial'
 'financial' 'creative']


In [24]:
import pickle

# Save the label encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)